In [ ]:
import pandas
import automol
import ioformat
import helpers

In [ ]:
# 1. read in the non-redundantly expanded species and reaction lists
NREX_C67_SPC_PATH = 'species/02_species-nr-exp_nuig-c6-7.csv'
NREX_C67_RXN_PATH = 'reactions/04_reactions-nr-exp_nuig-c6-7.csv'
NREX_PYR_SPC_PATH = 'species/02_species-nr-exp_nuig-pyro.csv'
NREX_PYR_RXN_PATH = 'reactions/04_reactions-nr-exp_nuig-pyro.csv'

NREX_C67_SPC_DF = pandas.read_csv(NREX_C67_SPC_PATH, quotechar="'")
NREX_C67_RXN_DF = pandas.read_csv(NREX_C67_RXN_PATH, quotechar="'")
NREX_PYR_SPC_DF = pandas.read_csv(NREX_PYR_SPC_PATH, quotechar="'")
NREX_PYR_RXN_DF = pandas.read_csv(NREX_PYR_RXN_PATH, quotechar="'")

In [ ]:
# 2. add missing non-canonical species enantiomers back into
#    non-redundant species list, adding columns containing
#    racemic ChIs and racemic CHEMKIN names
def add_missing_species_enantiomers(spc_df):
    row_dcts = []
    count = len(spc_df.index)
    for num, row0 in spc_df.iterrows():
        print(f'Row {num}/{count}')
        row_dct0 = row0.to_dict()
        name0 = row_dct0['name']
        chi0 = row_dct0['inchi']
        smi0 = row_dct0['smiles']
        # racemic name/ChI
        is_enant = automol.chi.is_enantiomer(chi0)
        namer = helpers.racemic_species_name(name0, chi0)
        chir = automol.chi.racemic(chi0)
        print(f'Species: {name0}')
        print(f' - ChI: {chi0}')
        print(f' - SMILES: {smi0}')
        row_dct0.update({
            'name': name0,
            'racem-name': namer,
            'racem-inchi': chir,
            'is_enant': is_enant,
        })
        row_dcts.append(row_dct0)
        if is_enant:
            print(f' - Species is chiral.')
            print(f' - Racemic name: {namer}')
            print(f' - Racemic ChI: {chir}')
            print(f' - Checking for missing enantiomer...')
            # name/ChI for other enantiomer
            name1 = helpers.reflect_species_name(name0, chi0)
            if name1 not in spc_df.index:
                chi1 = automol.chi.reflect(chi0)
                smi1 = automol.smiles.reflect(smi0)
                print(f' - Missing enantiomer: {name1}')
                print(f'   - ChI: {chi1}')
                print(f'   - SMILES: {smi1}')
                row_dct1 = row_dct0.copy()
                row_dct1.update({
                    'name': name1,
                    'inchi': chi1,
                    'smiles': smi1,
                    'is_enant': is_enant,
                })
                row_dcts.append(row_dct1)
    ex_spc_df = pandas.DataFrame.from_records(row_dcts)
    return ex_spc_df

In [ ]:
print('Retrieving full NUIG-C6-7 species expansion from non-redundant list')
FUEX_C67_SPC_DF = add_missing_species_enantiomers(NREX_C67_SPC_DF)

In [ ]:
print('Retrieving full NUIG-Pyro species expansion from non-redundant list')
FUEX_PYR_SPC_DF = add_missing_species_enantiomers(NREX_PYR_SPC_DF)

In [ ]:
# 2. add missing non-canonical reaction enantiomers back into
#    non-redundant reaction list, adding columns with racemic
#    CHEMKIN names
def add_missing_reaction_enantiomers(rxn_df, spc_df):
    """ Add missing enantiomers back into non-redundant raction list
    
        :param rxn_df: fully or non-redundantly expanded reaction list
        :param spc_df: fully expanded species list, with 'racem-name'
            and 'racem-inchi' columns added
    """
    row_dcts = []
    count = len(rxn_df.index)
    chi_dct = dict(zip(spc_df['name'], spc_df['inchi']))
    for num, row0 in rxn_df.iterrows():
        print(f'Row {num}/{count}')
        row_dct0 = row0.to_dict()
        name0 = row_dct0['name']
        print(f'Reaction: {name0}')
        namer = helpers.racemic_reaction_name(name0, chi_dct)
        name1 = helpers.reflect_reaction_name(name0, chi_dct)
        num_rct_enants = helpers.chiral_reactant_count(name0, chi_dct)
        row_dct0.update({
            'name': name0,
            'racem-name': namer,
            'num-reac-enants': num_rct_enants,
        })
        row_dcts.append(row_dct0)
        if name1 is not None:
            print(f' - Reaction is chiral.')
            print(f' - Racemic name: {namer}')
            if name1 not in rxn_df.index:
                print(f' - Missing enantiomer: {name1}')
                row_dct1 = row_dct0.copy()
                ts_amchi = row_dct0['ts_amchi_stereo']
                row_dct1.update({
                    'name': name1,
                    'num-reac-enants': num_rct_enants,
                    'ts_amchi_stereo': automol.chi.reflect(ts_amchi),
                })
                row_dcts.append(row_dct1)
    ex_rxn_df = pandas.DataFrame.from_records(row_dcts).set_index('name')
    ex_rxn_df.insert(0, 'racem-name', ex_rxn_df.pop('racem-name'))
    return ex_rxn_df

In [ ]:
print('Retrieving full NUIG-C6-7 reaction expansion from non-redundant list')
FUEX_C67_RXN_DF = add_missing_reaction_enantiomers(
    NREX_C67_RXN_DF, FUEX_C67_SPC_DF)

Row 2/5990
Reaction: I3C6Q25-1-ABB1 = I3C6D1-5OOH-AB1 + HO2
 - Reaction is chiral.
 - Racemic name: I3C6Q25-1-ABBr = I3C6D1-5OOH-ABr + HO2
 - Missing enantiomer: I3C6Q25-1-ABB0 = I3C6D1-5OOH-AB0 + HO2
Row 3/5990
Reaction: I3C6Q25-1-ABA1 = I3C6D1-5OOH-AA1 + HO2
 - Reaction is chiral.
 - Racemic name: I3C6Q25-1-ABAr = I3C6D1-5OOH-AAr + HO2
 - Missing enantiomer: I3C6Q25-1-ABA0 = I3C6D1-5OOH-AA0 + HO2
Row 4/5990
Reaction: I3C6-3 + H = I3C6
Row 5/5990
Reaction: I3C6Q16-4 = C4H71-4OOH + C2H4O2H
Row 6/5990
Reaction: NC5H12 + CH3O2 = C5H11-1 + CH3O2H
Row 7/5990
Reaction: IC6D4-3O-A0 = C2H3CHO + IC3H7
 - Reaction is chiral.
 - Racemic name: IC6D4-3O-Ar = C2H3CHO + IC3H7
 - Missing enantiomer: IC6D4-3O-A1 = C2H3CHO + IC3H7
Row 8/5990
Reaction: C6H12OOH1-3O2-A0 = C6H11Q13-5-A0
 - Reaction is chiral.
 - Racemic name: C6H12OOH1-3O2-Ar = C6H11Q13-5-Ar
 - Missing enantiomer: C6H12OOH1-3O2-A1 = C6H11Q13-5-A1
Row 9/5990
Reaction: C6H12OOH1-3O2-A1 = C6H11Q13-5-A1
 - Reaction is chiral.
 - Racemic name:

In [ ]:
print('Retrieving full NUIG-Pyro reaction expansion from non-redundant list')
FUEX_PYR_RXN_DF = add_missing_reaction_enantiomers(
    NREX_PYR_RXN_DF, FUEX_PYR_SPC_DF)

In [ ]:
# 4. construct a complete, non-redundant model by lumping enantiomers
def lump_enantiomers(rxn_df, spc_df):
    """ Lump species and reactions, assuming a racemic mixture
    """
    # First, lump species. This list is equivalent to the non-redundant species list,
    # with names replaced by racemic names
    lu_spc_df = spc_df.groupby('racem-name', sort=False).apply(lambda g: g.iloc[0])
    lu_spc_df.rename({'name': 'stereo-name'}, axis=1, inplace=True)
    lu_spc_df.index.name = 'name'
    lu_spc_df.pop('racem-name')
    lu_spc_df.reset_index(inplace=True)

    # Second, lump reactions. This is *not* equivalent to the non-redundant reaction list
    # (a.) Convert serialized parameter strings into objects
    rxn_df = helpers.reactions_with_params_objects(rxn_df)

    # (b.) Lump products, adding rate constants
    def _lump_reactions(grp_df):
        row = grp_df.iloc[0]
        row['stereo-name'] = grp_df.index[0]
        objs = grp_df['params']
        nreac = row['num-reac-enants']
        row['params'] = sum(objs) / (2 ** nreac)  # __radd__ is overloaded
        return row

    lu_rxn_df = rxn_df.groupby('racem-name', sort=False).apply(_lump_reactions)
    lu_rxn_df.index.name = 'name'
    lu_rxn_df.pop('racem-name')
    lu_rxn_df.insert(0, 'stereo-name', lu_rxn_df.pop('stereo-name'))
    lu_rxn_df.reset_index(inplace=True)
    return lu_rxn_df, lu_spc_df


LUMP_C67_RXN_DF, LUMP_C67_SPC_DF = lump_enantiomers(FUEX_C67_RXN_DF, FUEX_C67_SPC_DF)
LUMP_PYR_RXN_DF, LUMP_PYR_SPC_DF = lump_enantiomers(FUEX_PYR_RXN_DF, FUEX_PYR_SPC_DF)

LUMP_C67_RXN_PATH = 'reactions/06_reactions-lumped_nuig-c6-7.csv'
LUMP_C67_RXN_DF.to_csv(LUMP_C67_RXN_PATH, quotechar="'")
LUMP_PYR_RXN_PATH = 'reactions/06_reactions-lumped_nuig-pyro.csv'
LUMP_PYR_RXN_DF.to_csv(LUMP_PYR_RXN_PATH, quotechar="'")

In [ ]:
# 5. generate reaction parameters dictionaries
LUMP_C67_RXN_DCT = helpers.mechanism_dict(LUMP_C67_RXN_DF)
LUMP_PYR_RXN_DCT = helpers.mechanism_dict(LUMP_PYR_RXN_DF)

In [ ]:
# 6. restrict the species dataframes based on the reaction lists
LUMP_C67_SPC_DF = helpers.species_for_mechanism(LUMP_C67_SPC_DF, LUMP_C67_RXN_DCT)
LUMP_PYR_SPC_DF = helpers.species_for_mechanism(LUMP_PYR_SPC_DF, LUMP_PYR_RXN_DCT)

In [ ]:
# 7. write the original and fully expanded submechanisms
LUMP_C67_SPC_PATH = 'mechanisms/06_nr-exp_nuig-c6-7.csv'
LUMP_C67_RXN_PATH = 'mechanisms/06_nr-exp_nuig-c6-7.txt'
LUMP_PYR_SPC_PATH = 'mechanisms/06_nr-exp_nuig-pyro.csv'
LUMP_PYR_RXN_PATH = 'mechanisms/06_nr-exp_nuig-pyro.txt'

# a. write the species files
LUMP_C67_SPC_DF.to_csv(LUMP_C67_SPC_PATH, quotechar="'", index=False)
LUMP_PYR_SPC_DF.to_csv(LUMP_PYR_SPC_PATH, quotechar="'", index=False)

# b. write the reaction/mechanism files
LUMP_C67_MECH_STR = helpers.mechanism_string(LUMP_C67_RXN_DCT, LUMP_C67_SPC_DF)
LUMP_PYR_MECH_STR = helpers.mechanism_string(LUMP_PYR_RXN_DCT, LUMP_PYR_SPC_DF)

ioformat.pathtools.write_file(LUMP_C67_MECH_STR, '.', LUMP_C67_RXN_PATH)
ioformat.pathtools.write_file(LUMP_PYR_MECH_STR, '.', LUMP_PYR_RXN_PATH)